In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

driver = webdriver.Chrome()
driver.get("https://resultados.lamolina.edu.pe/resultados")
time.sleep(2)

# Click en "Buscar por carreras"
driver.find_element(By.XPATH, "//*[contains(text(), 'Buscar por carreras')]").click()
time.sleep(2)

# Lista de carreras
carreras = ['Agronomía', 'Biología', 'Ingeniería Ambiental', 'Ingeniería Meteorológica', 
           'Ingeniería Forestal', 'Economía', 'Estadística Informática', 'Ing. Gestión Empresarial',
           'Ingeniería Agrícola', 'Zootecnia', 'Pesquería', 'Ingeniería en Industrias Alimentarias']

todos_datos = []

for carrera in carreras:
    try:
        # Click en la carrera
        driver.find_element(By.XPATH, f"//*[contains(text(), '{carrera}')]").click()
        time.sleep(3)
        
        # Extraer datos de la tabla
        elementos = driver.find_elements(By.CSS_SELECTOR, "p.data-data")
        carrera_datos = []
        
        for i in range(0, len(elementos), 3):
            if i + 2 < len(elementos):
                nombre = elementos[i].text.strip()
                modalidad = elementos[i + 1].text.strip()
                omg = elementos[i + 2].text.strip()
                if nombre and omg.isdigit():
                    carrera_datos.append([nombre, modalidad, omg, carrera])
        
        todos_datos.extend(carrera_datos)
        print(f"✓ {carrera}: {len(carrera_datos)} ingresantes descargados")
        
        # Click en "Elegir otra carrera" en lugar de retroceder
        elegir_otra = driver.find_element(By.XPATH, "//*[contains(text(), 'Elegir otra carrera')]")
        elegir_otra.click()
        time.sleep(2)
        
    except:
        print(f"✗ Error con {carrera}")
        # Si hay error, intentar hacer click en "Elegir otra carrera"
        try:
            elegir_otra = driver.find_element(By.XPATH, "//*[contains(text(), 'Elegir otra carrera')]")
            elegir_otra.click()
            time.sleep(2)
        except:
            pass

# Guardar todo en un archivo
with open('todos_ingresantes.csv', 'w', newline='', encoding='utf-8') as archivo:
    writer = csv.writer(archivo)
    writer.writerow(['INGRESANTE', 'MODALIDAD', 'OMG', 'CARRERA'])
    writer.writerows(todos_datos)

print(f"Total: {len(todos_datos)} ingresantes de todas las carreras guardados")
driver.quit()

✓ Agronomía: 114 ingresantes descargados
✓ Biología: 23 ingresantes descargados
✓ Ingeniería Ambiental: 22 ingresantes descargados
✓ Ingeniería Meteorológica: 16 ingresantes descargados
✓ Ingeniería Forestal: 34 ingresantes descargados
✓ Economía: 35 ingresantes descargados
✓ Estadística Informática: 31 ingresantes descargados
✓ Ing. Gestión Empresarial: 39 ingresantes descargados
✓ Ingeniería Agrícola: 56 ingresantes descargados
✓ Zootecnia: 59 ingresantes descargados
✓ Pesquería: 32 ingresantes descargados
✓ Ingeniería en Industrias Alimentarias: 44 ingresantes descargados
Total: 505 ingresantes de todas las carreras guardados


In [1]:
import pandas as pd

# Leer archivo
df = pd.read_csv('todos_ingresantes.csv')

# Si todo está en una columna, separar por comas
if len(df.columns) == 1:
    df[['INGRESANTE', 'MODALIDAD', 'OMG']] = df.iloc[:,0].str.split(',', expand=True)

# Limpiar caracteres especiales que no son compatibles con latin-1
df = df.replace({
    '\u2013': '-',  # Guión largo
    '\u2014': '-',  # Guión em
    '\u2019': "'",  # Apostrofe curvo
    '\u201c': '"',  # Comilla izquierda
    '\u201d': '"'   # Comilla derecha
}, regex=True)

# Guardar para Excel
df.to_csv('ingresantes_UNALM_2025_II.csv', encoding='latin-1', sep=';', index=False, errors='replace')
print("✓ Convertido a ingresantes_UNALM_2025_II.csv")

✓ Convertido a ingresantes_UNALM_2025_II.csv
